In [ ]:
import json
import logging
import os
from pathlib import Path
import subprocess
import sys

from dotenv import load_dotenv
from IPython.display import Markdown, display
import mlflow
from mlflow import MlflowClient, MlflowException
from mlflow.deployments import get_deploy_client
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import ColSpec, ParamSchema, ParamSpec, Schema

In [ ]:
try:
    __file__  # NOQA: B018
except NameError:
    __file__ = __vsc_ipynb_file__  # NOQA: F821 # works for ipynb in vscode

repo = subprocess.check_output(  # NOQA: S603
    ["git", "rev-parse", "--show-toplevel"],  # NOQA: S607
    cwd=Path(__file__).parent,
    encoding="utf-8",
).strip()
repo = Path(repo).resolve()

datadir = Path(__file__).parent / "data"

In [ ]:
LOG_FMT = "%(asctime)s - %(levelname)-8s - %(name)s - %(funcName)s:%(lineno)d - %(message)s"

logging.basicConfig(format=LOG_FMT)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logging.getLogger("src").setLevel(logging.DEBUG)
logging.getLogger("mlflow").setLevel(logging.DEBUG)

In [ ]:
config = {"d": "docker", "k": "kubernetes"}[
    input(prompt="What configuration are we testing [docker, hybrid, kubernetes]: ").lower()[0]
]

if config == "docker":
    _ = load_dotenv(repo / "quickstart" / ".env", override=True)
    os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5555"
    # os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

elif config == "kubernetes":
    _ = load_dotenv(repo / ".env", override=True)


print(f"env vars for {config=}:")
print(os.environ["MLFLOW_TRACKING_URI"])
# print(os.environ["MLFLOW_S3_ENDPOINT_URL"])
# print(os.environ["AWS_ACCESS_KEY_ID"])
# print(os.environ["AWS_SECRET_ACCESS_KEY"])

In [ ]:
mlflow.set_tracking_uri(uri=os.environ["MLFLOW_TRACKING_URI"])

mlflow_experiment = "test-mlflow"
try:
    mlflow.set_experiment(mlflow_experiment)
except MlflowException:
    mlflow.create_experiment(mlflow_experiment)
    mlflow.set_experiment(mlflow_experiment)
print(mlflow.doctor())

In [ ]:
# mlflow.end_run()
with mlflow.start_run():
    print(mlflow.get_artifact_uri())
    mlflow.log_artifact(__file__)